In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from collections import Counter
import datetime
import time
import os
import json

from src.data import NSynthDataset
from src.utils import print_and_log
from src.models import Classifier

/opt/conda/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
if torch.cuda.is_available():  
    device = torch.device("cuda:0")
else:  
    device = torch.device("cpu") 

In [3]:
hparams = {
    'checkpoint' : None,
    'feature_type' : 'mel',
    'instrument_source' : [0, 1, 2],
    'remove_synth_lead' : True,
    'scaling' : 'normalize',
    'resize' : (128, 128),
    'n_samples_per_class' : None,
    'n_epochs' : 25,
    'batch_size' : 64,
    'lr' : 0.00001,
    'hidden_dim' : 1024,
    'display_iters' : 100,
    'val_iters' : 1000,
    'n_val_samples' : 1000, 
    'n_early_stopping' : 5 # stop if validation doesn't improve after this number of validation cycles
}

In [4]:
train_dataset = NSynthDataset(
    'data/nsynth/nsynth-train', 
    instrument_source=hparams['instrument_source'], 
    feature_type=hparams['feature_type'],
    scaling=hparams['scaling'],
    resize=hparams['resize'],
    include_meta=True,
    remove_synth_lead=hparams['remove_synth_lead'],
    n_samples_per_class=hparams['n_samples_per_class']
)

val_dataset = NSynthDataset(
    'data/nsynth/nsynth-valid', 
    instrument_source=hparams['instrument_source'], 
    feature_type=hparams['feature_type'],
    scaling=hparams['scaling'],
    resize=hparams['resize'],
    include_meta=True,
    remove_synth_lead=hparams['remove_synth_lead']
)

In [5]:
class_ctr = Counter([x['instrument_family_str'] for x in train_dataset.meta.values()])
class_dict = dict(enumerate(sorted(class_ctr)))
inv_class_dict = dict([(v, k) for k, v in class_dict.items()])
class_weights = np.array([max(class_ctr.values())/class_ctr[class_dict[i]] for i in range(len(class_dict))])

In [6]:
class_ctr

Counter({'guitar': 32690,
         'bass': 65474,
         'organ': 34477,
         'keyboard': 51821,
         'vocal': 10208,
         'string': 19474,
         'reed': 13911,
         'flute': 8773,
         'mallet': 34201,
         'brass': 12675})

In [7]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=hparams['batch_size'], shuffle=True)

In [8]:
model = Classifier(n_classes=len(class_dict), h_dim=hparams['hidden_dim']).to(device)
print('# of parameters : {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

# of parameters : 17587466


In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr=hparams['lr'])

In [10]:
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S')
results_dir = 'train_results/classifier/{}'.format(timestamp)
os.makedirs(results_dir)

with open(os.path.join(results_dir, 'hparams.json'), 'w') as fp:
    json.dump(hparams, fp)

log_file = os.path.join(results_dir, 'train_log.txt')
log = open(log_file, 'w')
log.close()
print_and_log('{}'.format(model.__class__.__name__), log_file)

Classifier


In [11]:
loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32, device=device))

ckpt_weights_path = None
best_loss = 1e10
since_best = 0
done = False

if hparams['checkpoint']:
    print_and_log('Resuming training from {}'.format(hparams['checkpoint']), log_file)
    ckpt = torch.load(hparams['checkpoint'])
    epoch = ckpt['epoch']
    itr = ckpt['itr']
    optimizer.load_state_dict(ckpt['optimizer'])
    model.load_state_dict(ckpt['model'])
    best_loss = ckpt['best_loss']
else:
    epoch = 0
    itr = 0

for epoch in range(epoch, hparams['n_epochs']):
    if done:
        break
        
    for batch in train_dataloader:
        if done:
            break
            
        itr += 1
        features = batch[0].unsqueeze(1).to(device)
        labels = torch.tensor([inv_class_dict[i] for i in batch[1]['instrument_family_str']], 
                              dtype=torch.long, device=device)
        outputs = model(features)
        
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (itr % hparams['display_iters'] == 0) or (itr == 1):
            acc = (outputs.argmax(-1) == labels).float().mean()
            print_and_log('[{}, {:5d}] loss : {:.4f}, acc : {:.4f}'.format(epoch, itr, loss.item(), acc.item()), log_file)
            
        if itr % hparams['val_iters'] == 0:
            model.eval()
            
            with torch.no_grad():
                ct, i, val_loss, val_acc = 0, 0, 0.0, 0.0
                for batch in val_dataloader:
                    i += 1
                    ct += batch[0].size(0)
                    features = batch[0].unsqueeze(1).to(device)
                    labels = torch.tensor([inv_class_dict[i] for i in batch[1]['instrument_family_str']], 
                                          dtype=torch.long, device=device)
                    outputs = model(features)

                    loss = loss_fn(outputs, labels)
                    acc = (outputs.argmax(-1) == labels).float().mean()

                    val_loss += (loss.item() - val_loss)/i
                    val_acc += (acc.item() - val_acc)/i

                    if ct >= hparams['n_val_samples']:
                        break
                
            print_and_log('Val - loss : {:.4f}, acc : {:.4f}'.format(val_loss, val_acc), log_file)
            
            if val_loss < best_loss:
                since_best = 0
                best_loss = val_loss
                
                # save weights
                if ckpt_weights_path:
                    os.remove(ckpt_weights_path)
                ckpt_weights_path = os.path.join(results_dir, 'model-{}.weights'.format(itr))
                torch.save(model.state_dict(), ckpt_weights_path)
                print_and_log('Weights saved in {}'.format(ckpt_weights_path), log_file)
                
                # save meta information
                ckpt_meta_path = os.path.join(results_dir, 'checkpoint')
                torch.save({
                    'best_loss' : best_loss,
                    'epoch' : epoch,
                    'itr' : itr,
                    'optimizer' : optimizer.state_dict(),
                    'model' : model.state_dict()
                }, ckpt_meta_path)
            else:
                since_best += 1
                if since_best >= hparams['n_early_stopping']:
                    done = True
                    print_and_log('Early stopping... training complete', log_file)
            
            model.train()

[0,     1] loss : 2.7466, acc : 0.0312
[0,   100] loss : 2.0178, acc : 0.2656
[0,   200] loss : 1.7442, acc : 0.3281
[0,   300] loss : 1.7187, acc : 0.4062
[0,   400] loss : 2.0157, acc : 0.2969
[0,   500] loss : 1.6467, acc : 0.2969
[0,   600] loss : 1.6640, acc : 0.4062
[0,   700] loss : 1.6781, acc : 0.3594
[0,   800] loss : 1.6506, acc : 0.3594
[0,   900] loss : 1.3026, acc : 0.4688
[0,  1000] loss : 1.5411, acc : 0.3906
Val - loss : 1.5279, acc : 0.4053
Weights saved in train_results/classifier/20200604-001402/model-1000.weights
[0,  1100] loss : 1.5964, acc : 0.4219
[0,  1200] loss : 1.3153, acc : 0.4688
[0,  1300] loss : 1.1297, acc : 0.5312
[0,  1400] loss : 1.3688, acc : 0.3906
[0,  1500] loss : 1.1943, acc : 0.5156
[0,  1600] loss : 1.5572, acc : 0.3906
[0,  1700] loss : 1.2165, acc : 0.4844
[0,  1800] loss : 1.2623, acc : 0.5938
[0,  1900] loss : 1.3649, acc : 0.4844
[0,  2000] loss : 1.0395, acc : 0.5781
Val - loss : 1.3799, acc : 0.4795
Weights saved in train_results/class